In [1]:
import pandas as pd
import numpy as np
import warnings
import tkinter as tk
warnings.filterwarnings('ignore')

In [4]:
cols = 'user_ID Item_ID rating timestamp'.split()
dat=pd.read_csv('u.data',sep='\t',names=cols)
mcol=['Item_ID','title']+[str(i) for i in range(22)]
mdat=pd.read_csv('u.item',sep="|",names=mcol)
mdat=mdat[['Item_ID','title']]
det=pd.merge(dat,mdat,on='Item_ID')
rat=pd.DataFrame(det.groupby('title')['rating'].mean())
rat['count']=det['title'].value_counts()
pvt=det.pivot_table(index='user_ID', columns='title',values='rating')



gh=tk.Tk()
gh.title('Recomendation system')
gh.geometry('500x400')
gh.configure(bg='#769c96')


tk.Label(gh,text='Movie watched',bg='#239999').place(x=20,y=60)
tk.Label(gh,text='Recomended to you',bg='#239999').place(x=20,y=250)
tk.Label(gh,text='Also watch',bg='#239999').place(x=20,y=300)

mv=tk.Variable(gh)
rec=tk.Variable(gh)
als=tk.Variable(gh)

rec.set('type movie name')
als.set('type movie name')

tk.Entry(gh,textvariable=mv,bg='#239999',width=40).place(x=150,y=60)
tk.Label(gh,textvariable=rec,bg='#239999',width=40).place(x=150,y=250)
tk.Label(gh,textvariable=als,bg='#239999',width=40).place(x=150,y=300)


def fiind():
    ab=mv.get().lower().strip()
    if ab:
        try:
            ab=mdat['title'][mdat['title'].apply(lambda x: ab in x.lower())].values[0]
            mv.set(ab)
        except IndexError:
            rec.set('movie not found')
            als.set('movie not found')
        else:    
            abc=pvt[ab]
            qwe=pvt.corrwith(abc)
            ert=pd.DataFrame(qwe,columns=['correlation'])
            ert.dropna(inplace=True)
            ert=ert.join(rat['count'])
            rec.set(ert[(ert['count']>150)&(ert['correlation']>0.5)].sort_values(by='correlation',ascending=False).index[1])
            als.set(ert[ert['correlation']>0.5].sort_values(by='correlation',ascending=False).index[0])    
    
    
tk.Button(gh,text='Find Recommends',command=fiind,bg='#20c7c7').place(x=180,y=150)



gh.mainloop()
